In [ ]:

import fiona
import geopandas as gpd
from matplotlib import pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import pathlib
import shapely
from scipy import stats
from tqdm.notebook import tqdm

mpl.rcParams['figure.dpi'] = 144
mpl.style.use('scrartcl.mplstyle')

In [ ]:
# Load all the data we need.
ROOT = pathlib.Path('data/wastewater_catchment_areas_public')

lsoas = gpd.read_file('data/geoportal.statistics.gov.uk/LSOA11_BGC.zip').set_index('LSOA11CD')

catchments = gpd.read_file(ROOT / 'catchments_consolidated.shp')

lsoa_catchment_lookup = pd.read_csv(ROOT / 'lsoa_catchment_lookup.csv')

lsoa_coverage = pd.read_csv(ROOT / 'lsoa_coverage.csv')

lsoa_population = pd.read_csv('data/ons.gov.uk/lsoa_syoa_all_years_t.csv',
                              usecols=['LSOA11CD', 'year', 'Pop_Total'])
lsoa_population['year'] = lsoa_population.year.apply(lambda x: int(x[4:]))

waterbase_catchment_lookup = pd.read_csv(ROOT / 'waterbase_catchment_lookup.csv')

waterbase_consolidated = pd.read_csv(ROOT / 'waterbase_consolidated.csv', 
                                     index_col=['uwwCode', 'year'])
# Fix a data problem where someone dropped a zero (or another digit) for Kinmel Bay.
waterbase_consolidated.loc[('UKWAWA_WW_TP000093', 2016), 'uwwLoadEnteringUWWTP'] *= 10

# Add up the treated load for the two works in Abingdon (which should really just be one).
x = waterbase_consolidated.loc['UKENTH_TWU_TP000001'].uwwLoadEnteringUWWTP
y = waterbase_consolidated.loc['UKENTH_TWU_TP000165'].uwwLoadEnteringUWWTP
z = y.reindex(x.index).fillna(0) + x
waterbase_consolidated.loc['UKENTH_TWU_TP000001', 'uwwLoadEnteringUWWTP'] = z.values

# Get rid of the duplicate treatment work.
waterbase_consolidated = waterbase_consolidated.drop('UKENTH_TWU_TP000165', level=0)
waterbase_consolidated = waterbase_consolidated.reset_index()

In [ ]:
# Evaluate the total intersection area for each LSOA.
intersection_area_sum = lsoa_catchment_lookup.groupby('LSOA11CD')\
    .intersection_area.sum().reset_index(name='intersection_area_sum')

# Construct a data frame that has a number of different areas that we can use for normalisation.
merged = pd.merge(lsoa_coverage, intersection_area_sum, on='LSOA11CD')
merged = pd.merge(merged, lsoa_catchment_lookup, on='LSOA11CD')
merged = pd.merge(merged, lsoa_population, on='LSOA11CD')

def aggregate(subset):
    # Construct different normalisations.
    norms = {
        'norm_total_area': subset.total_area,
        'norm_area_covered': subset.area_covered,
        'norm_intersection_sum': subset.intersection_area_sum,
    }
    intersection_area_pop = subset.intersection_area * subset.Pop_Total
    return pd.Series({key: (intersection_area_pop / value).sum() for key, value in norms.items()})

grouped = merged.groupby(['identifier', 'year'])
geospatial_estimate = grouped.apply(aggregate)
geospatial_estimate.head()

In [ ]:
# Show population estimates by company.
merged = pd.merge(catchments, geospatial_estimate.reset_index(), on=['identifier'])
totals = merged[merged.year == 2016].groupby('company').norm_area_covered.sum()
print(f'total population served: {totals.sum() / 1e6:.3f}m')
totals / 1e6

In [ ]:
# Merge the waterbase data (BOD p.e.) with geospatial population estimates for comparison.
merged = pd.merge(waterbase_catchment_lookup, waterbase_consolidated, on=['uwwCode', 'uwwName'])
merged = pd.merge(merged, geospatial_estimate, on=['year', 'identifier'])

# Sum by year and uwwCode (because the same treatment work may be linked to multiple catchments if 
# the subcatchment aggregation didn't work out properly). Then assign back to the merged dataset and 
# drop duplicates.
estimates = merged.groupby(['uwwCode', 'year']).agg({
    'norm_total_area': 'sum',
    'norm_area_covered': 'sum',
})
for key in estimates:
    merged[key] = [estimates.loc[(x.uwwCode, x.year), key] for _, x in merged.iterrows()]
merged = merged.drop_duplicates(['uwwCode', 'year'])

# Drop treatment works for Scotland and Southwest water because we don't have 
# LSOAs and catchments for them, respectively.
merged = merged[~merged.uwwCode.str.startswith('UKSC')]
merged = merged[~merged.uwwCode.str.startswith('UKENSW_SWS')]

# Evaluate the pearson correlation on the log scale (omitting treatment works without load).
f = merged.uwwLoadEnteringUWWTP > 0
stats.pearsonr(np.log(merged.uwwLoadEnteringUWWTP[f]), np.log(merged.norm_area_covered[f]))

In [ ]:
# Show a figure of different population estimates for a given year.
fig = plt.figure()
gs = fig.add_gridspec(2, 2)
ax = fig.add_subplot(gs[:, 0])
year = 2016
subset = merged[merged.year == year]
ax.scatter(subset.uwwLoadEnteringUWWTP, subset.norm_area_covered, marker='.', alpha=.5)
ax.set_yscale('log')
ax.set_xscale('log')
lims = subset.uwwLoadEnteringUWWTP.quantile([0, 1])
ax.plot(lims, lims, color='k', ls=':')
ax.set_aspect('equal')
ax.set_xlabel('BOD person equivalent')
ax.set_ylabel('Geospatial population estimate')
ax.text(0.05, 0.95, '(a)', transform=ax.transAxes, va='top')

# Annotations.
annotations = [
    {
        'code': 'UKENNE_NU_TP000026', 
        'label': 'Haggerston',
        'xfactor': 3,
        'yfactor': 1,
    },
    {
        'code': 'UKWAWA_WW_TP000016',
        'label': 'Rotherwas',
        'xfactor': 2.5,
    },
    {
        'code': 'UKENAN_AW_TP000020',
        'label': 'Billericay',
        'xfactor': 2/3,
        'yfactor': 3,
        'kwargs': {'ha': 'center'},
    },
    {
        'code': 'UKENAN_AW_TP000051',
        'label': 'Chalton',
        'xfactor': 1 / 3,
        'kwargs': {'ha': 'right'},
    },
]
indexed = subset.set_index('uwwCode')
for annotation in annotations:
    item = indexed.loc[annotation['code']]

    ax.annotate(
        annotation['label'], 
        (item.uwwLoadEnteringUWWTP, item.norm_area_covered),
        (item.uwwLoadEnteringUWWTP * annotation.get('xfactor', 1),
            item.norm_area_covered * annotation.get('yfactor', 1)),
        arrowprops={
            'arrowstyle': '-|>',
        },
        va='center', 
        **annotation.get('kwargs', {}),
    )
    print(annotation['label'], item.uwwName)

ax3 = ax = fig.add_subplot(gs[:, 1])
target = lambda x: np.median(np.abs(np.log10(x.norm_area_covered / x.uwwLoadEnteringUWWTP)))

x = []
y = []
ys = []
for year, subset in tqdm(merged.groupby('year')):
    x.append(year)
    # Evaluate the statistic.
    y.append(target(subset))
    # Run a bootstrap sample.
    ys.append([target(subset.iloc[np.random.randint(len(subset), size=len(subset))]) 
               for _ in range(1000)])
    
ys = np.asarray(ys)
l, u = np.percentile(ys, [25, 75], axis=1)
ax.errorbar(x, y, (y - l, u - y), marker='.')
ax.ticklabel_format(scilimits=(0, 0), axis='y', useMathText=True)
ax.set_xlabel('Year')
ax.set_ylabel('Median absolute\n$\\log_{10}$ error')
ax.xaxis.set_ticks([2006, 2008, 2010, 2012, 2014, 2016])
plt.setp(ax.xaxis.get_ticklabels(), rotation=30, ha='right')
ax.text(0.95, 0.95, '(b)', transform=ax.transAxes, ha='right', va='top')

fig.tight_layout()
fig.savefig('population-estimates.pdf')

# Show the log10 median absolute error over time.
y

In [ ]:
# Plot to illustrate why we're using area covered.
fig, ax = plt.subplots()

xmin = 515000
xmax = 523000
ymin = 170000
ymax = 176000
box = shapely.geometry.box(xmin, ymin, xmax, ymax)

# Plot the catchments.
idx_catchment = catchments.sindex.query(box)
catchments.iloc[idx_catchment].intersection(box).plot(ax=ax, color=['C0', 'k', 'C1', 'C2'])

# Plot the LSOAs.
idx = lsoas.sindex.query(box)
lsoas.iloc[idx].plot(ax=ax, facecolor='none', edgecolor='k', alpha=.1)
lsoas.loc[['E01003817']].plot(ax=ax, facecolor=(.5, .5, .5, .25), edgecolor='k')

ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_axis_off()
handles = [mpl.patches.Rectangle((0, 0), 1, 1, color=f'C{i}')
           for i in range(3)]
labels = ['Mogden', 'Hogsmill', 'Crossness']
ax.legend(handles, labels)

fig.tight_layout()
fig.savefig('estimation_method.pdf')